In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder \
    .appName('Read CSV File into DataFrame') \
    .config('spark.executor.memory', '10g') \
    .config('spark.driver.memory', '10g') \
    .getOrCreate()
# spark = SparkSession.builder.appName('Read CSV File into DataFrame').getOrCreate()
from pyspark.sql.functions import col

df=spark.read.csv("/home/nirmala/Documents/precog_requirement_task/cases/cases/cases_2012.csv" ,header=True)
df2=pd.read_csv('/home/nirmala/2012_filtered.csv')

In [3]:
df_pend=df
df_pend = df_pend.drop(col("dist_code"))
df_pend = df_pend.drop(col("ddl_case_id"))
df_pend = df_pend.drop(col("court_no"))
df_pend = df_pend.drop(col("cino"))
df_pend = df_pend.drop(col("judge_position"))
df_pend = df_pend.drop(col("female_defendant"))
df_pend = df_pend.drop(col("female_petitioner"))
df_pend = df_pend.drop(col("female_adv_def"))
df_pend = df_pend.drop(col("female_adv_pet"))
df_pend = df_pend.drop(col("type_name"))
df_pend = df_pend.drop(col("purpose_name"))
df_pend = df_pend.drop(col("disp_name"))
df_pend = df_pend.drop(col("date_last_list"))
df_pend = df_pend.drop(col("date_first_list"))

In [4]:
# del dp


In [5]:
df = df_pend.toPandas()

In [6]:
# del df_pend

In [7]:
df['date_of_filing'] = pd.to_datetime(df['date_of_filing'], format='%Y-%m-%d',errors = 'coerce') 
df['date_of_decision'] = pd.to_datetime(df['date_of_decision'], format='%Y-%m-%d',errors = 'coerce') 
df['date_next_list'] = pd.to_datetime(df['date_next_list'], format='%Y-%m-%d',errors = 'coerce') 
# df['file_end'] =(df['date_next_list']-df['date_of_filing']).dt.year

In [8]:
df['closing'] = df['date_next_list'].dt.year
df['filing'] = df['date_of_filing'].dt.year
df=df.drop(columns=['date_of_filing','date_of_decision','date_next_list'])

In [9]:
df['closing']=df['closing'].fillna('1900')
df['closing']=df['closing'].astype(int)
df['filing']=df['filing'].astype(int)

In [10]:
df = df[~df['closing'].isin([2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022])]
df['pending_years']=df['closing']-df['filing']
df=df.drop(columns=['filing','closing'])
df=df.reset_index()
df=df.drop(columns=['index'])

In [11]:
df['state_code'].replace(to_replace='01', value="Maharashtra",inplace=True)
df['state_code'].replace(to_replace='02' ,value="Andhra Pradesh",inplace=True)
df['state_code'].replace(to_replace='03' ,value="Karnataka",inplace=True)
df['state_code'].replace(to_replace='04', value="Kerala",inplace=True)
df['state_code'].replace(to_replace='05' ,value="Himachal Pradesh",inplace=True)
df['state_code'].replace(to_replace='06', value="Assam",inplace=True)
df['state_code'].replace(to_replace='07', value="Jharkhand",inplace=True)
df['state_code'].replace(to_replace='08' ,value="Bihar",inplace=True)
df['state_code'].replace(to_replace='09' ,value="Rajasthan",inplace=True)
df['state_code'].replace(to_replace='10' ,value="Tamilnadu",inplace=True)
df['state_code'].replace(to_replace='11',value="Orissa",inplace=True)
df['state_code'].replace(to_replace='12' ,value="Jammu and kashmir",inplace=True)
df['state_code'].replace(to_replace='13' ,value="Uttar pradesh",inplace=True)
df['state_code'].replace(to_replace='14' ,value="Haryana",inplace=True)
df['state_code'].replace(to_replace='15' ,value="Uttarakhand",inplace=True)
df['state_code'].replace(to_replace='16' ,value="West Bengal",inplace=True)
df['state_code'].replace(to_replace='17' ,value="Gujarath",inplace=True)
df['state_code'].replace(to_replace='18' ,value="Chattisgarh",inplace=True)
df['state_code'].replace(to_replace='19' ,value="Mizoram",inplace=True)
df['state_code'].replace(to_replace='20' ,value="Tripura",inplace=True)
df['state_code'].replace(to_replace='21' ,value="Meghalaya",inplace=True)
df['state_code'].replace(to_replace='22' ,value="Punjab",inplace=True)
df['state_code'].replace(to_replace='23' ,value="Madhya Pradesh",inplace=True)
df['state_code'].replace(to_replace='24' , value="Sikkim",inplace=True)
df['state_code'].replace(to_replace='25' ,value="Manipur",inplace=True)
df['state_code'].replace(to_replace='26' ,value="Delhi",inplace=True)
df['state_code'].replace(to_replace='27' ,value="Chandigarh",inplace=True)
df['state_code'].replace(to_replace='29' ,value="Telangana",inplace=True)
df['state_code'].replace(to_replace='30' ,value="Goa",inplace=True)
df['state_code'].replace(to_replace='31' ,value="Diu and daman",inplace=True)
df['state_code'].replace(to_replace='32',value="DNH at silvasa",inplace=True)
df['state_code'].replace(to_replace='33' ,value="Ladakh",inplace=True)


In [12]:
df_pending = df["state_code"].value_counts().reset_index()
df_pending['state_code']=df_pending['state_code'].sort_values()
df_pending.rename(columns={"index": "state", "state_code": "pendingcases_count"}, inplace=True)
new_rows_pending = pd.DataFrame({'state' : ['Diu and daman','Ladakh','DNH at silvasa','Manipur','Meghalaya','Mizoram','Sikkim'], 'pendingcases_count' : [0,0,0,0,0,0,0]})
df_pending= pd.concat([new_rows_pending, df_pending], ignore_index=True)
df_pending['pending_casesperlakh']=df_pending['pendingcases_count']/100000
df_pending.insert(3, 'Year', 2012)

In [13]:
df_pending=df_pending.sort_values(by='state')
df_pending=df_pending.reset_index()
df_pending=df_pending.drop(columns=['index'])  

In [14]:
df_pending.info()
df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   state                 32 non-null     object 
 1   pendingcases_count    32 non-null     int64  
 2   pending_casesperlakh  32 non-null     float64
 3   Year                  32 non-null     int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 1.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   year                            32 non-null     int64  
 1   state                           32 non-null     object 
 2   dismissed_count                 32 non-null     int64  
 3   prison_count                    32 non-null     int64  
 4   withdrawal_count                32 non-null     int64  


In [15]:
df_pending

,state,pendingcases_count,pending_casesperlakh,Year
0,Andhra Pradesh,838,0.00838,2012
1,Assam,900,0.00900,2012
2,Bihar,1237,0.01237,2012
3,Chandigarh,9,0.00009,2012
4,Chattisgarh,255,0.00255,2012
5,DNH at silvasa,0,0.00000,2012
6,Delhi,365,0.00365,2012
7,Diu and daman,0,0.00000,2012
8,Goa,4931,0.04931,2012
9,Gujarath,4234,0.04234,2012


In [17]:
df_merged=pd.merge(df2,df_pending,on='state')
df_merged

,year,state,dismissed_count,prison_count,withdrawal_count,avg_decision_end_per1lakhcases,avg_fileend_per1lakhcases,avg_filecourt_per1lakhcases,pendingcases_count_x,pending_casesperlakh_x,criminal_count,criminal_perlakh,noncriminal_count,noncriminal_perlakh,pendingcases_count_y,pending_casesperlakh_y,Year
0,2012,Andhra Pradesh,40361,0,2068,0,503,114,838,0.00838,123052,1.23052,72407,0.72407,838,0.00838,2012
1,2012,Assam,12621,422,1183,2,953,329,900,0.00900,56425,0.56425,77753,0.77753,900,0.00900,2012
2,2012,Bihar,22569,1,1413,8,889,419,1237,0.01237,97259,0.97259,188089,1.88089,1237,0.01237,2012
3,2012,Chandigarh,3642,0,3959,75,3146,579,9,0.00009,18866,0.18866,16193,0.16193,9,0.00009,2012
4,2012,Chattisgarh,9624,0,1279,2,1474,382,255,0.00255,45705,0.45705,42401,0.42401,255,0.00255,2012
5,2012,DNH at silvasa,66,0,66,1,108045,5640,0,0.00000,456,0.00456,599,0.00599,0,0.00000,2012
6,2012,Delhi,18030,0,18388,7,749,85,365,0.00365,99364,0.99364,75430,0.75430,365,0.00365,2012
7,2012,Diu and daman,99,0,96,0,30886,3536,0,0.00000,1021,0.01021,342,0.00342,0,0.00000,2012
8,2012,Goa,26,0,88,83,2403,502,4931,0.04931,24438,0.24438,20889,0.20889,4931,0.04931,2012
9,2012,Gujarath,30255,0,35241,0,202,68,4234,0.04234,330509,3.30509,201861,2.01861,4234,0.04234,2012


In [16]:
df_merged=df_merged.drop(columns=['Year'])

NameError: name 'df_merged' is not defined

In [28]:
df_merged.to_csv('2012_filtered.csv', index=False)


In [35]:
df_merged=df_merged.reindex(columns=['year','state','dismissed_count','prison_count','withdrawal_count','avg_decision_end_per1lakhcases','avg_fileend_per1lakhcases','avg_filecourt_per1lakhcases','pendingcases_count','pending_casesperlakh','criminal_count','criminal_perlakh','noncriminal_count','noncriminal_perlakh'])

In [36]:
df_merged.to_csv('2012_filtered.csv', index=False)


In [32]:
del df_merged